In [2]:
import arviz as az
import numpy as np
"""
# If pymc3
"""
# import pymc3 as pm
# from theano import tensor as tt
"""
# If pymc4
"""
import pymc as pm
import pytensor.tensor as tt

import random
import os
import sys
import matplotlib.pyplot as plt

from dataloader import *
sys.path.append('../')

from os import path

import warnings
warnings.filterwarnings("ignore")

np.random.seed(1116)

def Bayesian_IDM_pool(vt, s, dv, label_v):
    print("training size:", label_v.shape[0])
    dt = 1 / 30 * FREQ

    model = pm.Model()

    D = 5
    
    with model:
        def IDM_v(VMAX, DSAFE, TSAFE, AMAX, AMIN, DELTA, s, vt, dv):
            sn = DSAFE + vt * TSAFE + vt * dv / (2 * np.sqrt(AMAX * AMIN))
            a = AMAX * (1 - (vt / VMAX) ** DELTA - (sn / s) ** 2)
            return vt + a * dt
        mu_prior = pm.floatX(np.array([0, 0,0,0,0]))
        parameters_normalized = pm.MvNormal('mu_normalized', mu_prior, chol=np.eye(D))
        
        log_parameters = pm.Deterministic('log_mu', parameters_normalized*np.array([.3, 1., 1., .01, .5])
                                      +np.array([2., 0.69, 0.47, -.3, -.51]))
        parameters = pm.Deterministic('mu', tt.exp(log_parameters))
        
        DELTA = 4
        
        log_s_v = pm.Uniform('log_s_v', lower=-5.0, upper=-1.0)
        s_v = pm.Deterministic('s_v', tt.exp(log_s_v))
        
        v_obs = pm.Normal('obs', mu=IDM_v(parameters[0], parameters[1], parameters[2], parameters[3],
                                          parameters[4], DELTA, s, vt, dv), sigma=s_v, observed=label_v)

        tr = pm.sample(5000, tune=20000, random_seed=16, init='jitter+adapt_diag_grad', chains=2,
                       cores=8, discard_tuned_samples=True, return_inferencedata=True, target_accept=0.95)
    return tr, model

In [3]:
data = pd.DataFrame()
for file in os.listdir('Alafaya'):
    if file.endswith('.csv'):
        data = data.append(pd.read_csv('Alafaya/University@Alafaya-01.csv'))
data = data.reset_index()
data = data[data['frameNum'].apply(lambda x: x % FREQ == 0)]
# pts = gpd.GeoDataFrame(geometry=data[['carCenterLon', 'carCenterLat']].apply(lambda x: Point(x), axis=1)).set_crs({'init': 'epsg:4326'}).to_crs({'init': 'epsg:3857'})
# data[['x', 'y']] = pts.apply(lambda x: [x.geometry.x, x.geometry.y], axis=1)
data = extract_info(data)

In [ ]:
import pandas as pd

# data = pd.read_csv('results.csv')

new_data = []
ratio = 0.44704 # convert from mph to m/s
for index, group in data.groupby(['carId']):
    group = group.sort_values(by=['frameNum'])
    group['next_speed'] = group['speed'].shift(-1) * ratio
    group['speed'] = group['speed'] * ratio
    group['relative_speed'] = group['relative_speed'] * ratio
    new_data.append(group)
new_data = pd.concat(new_data)
new_data.dropna(inplace=True)
new_data = new_data[(new_data.speed > 0)]


speed = new_data['speed'].values
relative_speed = new_data['relative_speed'].values
distance = new_data['distance'].values 
next_speed = new_data['next_speed'].values

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = False
new_data['relative_speed'].hist()

In [ ]:
tr, model = Bayesian_IDM_pool(speed, distance, relative_speed, next_speed)

In [ ]:
az.summary(tr, var_names=["mu","log_mu","s_v"])

In [ ]:
df = tr.posterior.mu.to_dataframe()
results = pd.DataFrame()
for i in range(5):
    col = df.iloc[df.index.get_level_values('mu_dim_0') == i, 0]
    results['mu_{}'.format(i)] = list(col)

In [ ]:
results.mean(axis=0)

In [ ]:
results.cov()